In [18]:
class Util:
    @staticmethod
    def to_binary(num, len):
        return format(num, '0' + str(len) + 'b')
    def find_public(sv, fv):
        for i, vi in enumerate(sv):
            for j, vj in enumerate(fv):
                if vi == vj:
                    return i, j
        return -1,-1

In [19]:
from copy import deepcopy

In [20]:
class Node:
    def __init__(self, name, var_list):
        self.name = name
        self.var_list = var_list
        self.cpt = {}
    def setCpt(self, cpt):
        self.cpt = cpt
    def printInf(self):
        print("Name: " + self.name)
        print(" vars: " + str(self.var_list))
        for k in self.cpt:
            print('   key: ' + k + ' val: ' + str(self.cpt[k]))
        print('')
    def multiply(self, factor):
        # 查找要消去的变量在两个因子中的位置
        si, fi = Util.find_public(self.var_list, factor.var_list)
        
        # 构造相乘后新因子涉及的变量列表
        new_list = deepcopy(self.var_list)
        factor_list_copy = deepcopy(factor.var_list)
        del factor_list_copy[fi]
        new_list = new_list + factor_list_copy

        new_cpt = {}
        # 遍历两个因子的条件概率表中的所有条件概率
        for sk, sv in self.cpt.items():
            for fk, fv in factor.cpt.items():
                # 如果两个条件概率中要消去的变量的值相同
                if sk[si] == fk[fi]:
                    # 构造新的条件概率
                    new_k = deepcopy(sk)
                    fk_copy = deepcopy(fk)
                    del fk_copy[fi]
                    new_k = new_k + fk_copy
                    if new_k in new_cpt:
                        new_cpt[new_k] += sv * fv
                    else:
                        new_cpt[new_k] = sv * fv
        new_node = Node('f' + str(new_list), new_list)
        new_node.setCpt(new_cpt)
        return new_node
    def sumout(self, var):
        # 新因子的列表中删除var
        new_list = deepcopy(self.var_list)
        new_list.remove(var)
        
        new_cpt = {}
        for sk, sv in self.cpt.items():
            new_k = deepcopy(sk)
            new_k.remove()
        new_node = Node('f' + str(new_list), new_list)
        new_node.setCpt(new_cpt)
        return new_node
    def restrict(self, var, val):
        #TODO
        new_node = Node('f' + str(new_list), new_list)
        new_node.setCpt(new_cpt)
        return new_node

In [11]:
class VE:
    @staticmethod
    def inference(F, Q, R, E):
        # 遍历所有因子，用证据变量进行约束
        for f in F:
            for var, val in E.items():
                if var in f.var_list:
                    f.restrict(var, val)
        # 按消元顺序遍历所有剩余变量
        for v in R:
            # 筛选涉及该剩余变量的因子
            invole_v = []
            for idx, f in enumerate(F):
                if v in f.var_list:
                    invole_v.append(f)
                    del F[idx]
            # 将所有找到的因子相乘
            g = invole_v[0]
            for f in invole_v[1:]:
                g = g.multiply(f)
            # 相乘得到的新的因子加入F
            F.append(g)
        print("RESULT:")
        # 将所有剩余的因子相乘（这些因子只涉及查询变量）
        res = F[0]
        for f in F[1:]:
            res = res.multiply(f)
        total = sum(res.cpt.values())
        res.cpt={k:v/total for k, v in res.cpt.items()}
        res.printInf()
    @staticmethod
    def printFactors(F):
        for f in F:
            f.printInf()

In [13]:
B = Node('B', ['B'])
E = Node('E', ['E'])
A = Node('A', ['A', 'B', 'E'])
J = Node('J', ['J', 'A'])
M = Node('M', ['M', 'A'])

In [15]:
B.setCpt({'0': 0.999, '1': 0.001})
E.setCpt({'0': 0.998, '1': 0.002})
A.setCpt({'111': 0.95, '011': 0.05, '110':0.94, '010':0.06,
          '101': 0.29, '001': 0.71, '100':0.001,'000':0.999})
J.setCpt({'11':0.90, '01': 0.10, '10': 0.05, '00': 0.95})
M.setCpt({'11':0.70, '01': 0.30, '10': 0.01, '00': 0.99})

In [48]:
class Util:
    @staticmethod
    def to_binary(num, len):
        return format(num, '0' + str(len) + 'b')
    def find_public(sv, fv):
        for i, vi in enumerate(sv):
            for j, vj in enumerate(fv):
                if vi == vj:
                    return i, j
        return -1,-1

from copy import deepcopy


class Node:
    def __init__(self, name, var_list):
        self.name = name
        self.var_list = var_list
        self.cpt = {}

    def setCpt(self, cpt):
        self.cpt = cpt

    def printInf(self):
        print("Name: " + self.name)
        print(" vars: " + str(self.var_list))
        for k in self.cpt:
            print('   key: ' + k + ' val: ' + str(self.cpt[k]))
        print('')

    def multiply(self, factor):
        # 查找要消去的变量在两个因子中的位置
        si, fi = Util.find_public(self.var_list, factor.var_list)

        # 构造相乘后新因子涉及的变量列表
        new_list = deepcopy(self.var_list)
        factor_list_copy = deepcopy(factor.var_list)
        del factor_list_copy[fi]
        new_list = new_list + factor_list_copy

        new_cpt = {}
        # 遍历两个因子的条件概率表中的所有条件概率
        for sk, sv in self.cpt.items():
            for fk, fv in factor.cpt.items():
                # 如果两个条件概率中要消去的变量的值相同
                if sk[si] == fk[fi]:
                    # 构造新的条件概率
                    new_k = deepcopy(sk)
                    fk_copy = deepcopy(fk)
                    if fi == len(fk_copy):
                        fk_copy = fk_copy[:fi-1]
                    else:
                        fk_copy = fk_copy[:fi] + fk_copy[(fi+1):]
                    new_k = new_k + fk_copy
                    if new_k in new_cpt:
                        new_cpt[new_k] += sv * fv
                    else:
                        new_cpt[new_k] = sv * fv
        new_node = Node('f' + str(new_list), new_list)
        new_node.setCpt(new_cpt)
        return new_node

    def sumout(self, var):
        # 新因子的变量列表中删除var
        new_list = deepcopy(self.var_list)
        si = 0
        for i in range(len(new_list)):
            if new_list[i] == var:
                si = i
                break
        new_list.remove(var)
        new_cpt = {}
        # 将删除变量var后条件概率表中相同的表项相加
        for sk, sv in self.cpt.items():
            new_k = deepcopy(sk)
            if si == len(new_k):
                new_k = new_k[:si-1]
            else:
                new_k = new_k[:si] + new_k[(si+1):]
            if new_k in new_cpt:
                new_cpt[new_k] += sv
            else:
                new_cpt[new_k] = sv
        new_node = Node('f' + str(new_list), new_list)
        new_node.setCpt(new_cpt)
        return new_node

    def restrict(self, var, val):
        # 新因子的变量列表中删除var
        new_list = deepcopy(self.var_list)
        si = 0
        for i in range(len(new_list)):
            if new_list[i] == var:
                si = i
                break
        new_list.remove(var)
        # 删除所有原来因子的条件变量表中var值不是val的条目
        new_cpt = {}
        idx = 0
        for i, v in enumerate(self.var_list):
            if v == var:
                idx = i
                break
        for sk, sv in self.cpt.items():
            if sk[idx] == val:
                new_k = deepcopy(sk)
                if si == len(new_k):
                    new_k = new_k[:si-1]
                else:
                    new_k = new_k[:si] + new_k[(si+1):]
                new_cpt[new_k] = sv
        new_node = Node('f' + str(new_list), new_list)
        new_node.setCpt(new_cpt)
        return new_node

class VE:
    @staticmethod
    def inference(F, Q, R, E):
        # 遍历所有因子，用证据变量进行约束
        for f in F:
            for var, val in E.items():
                if var in f.var_list:
                    f.restrict(var, val)
        # 按消元顺序遍历所有剩余变量
        for v in R:
            # 筛选涉及该剩余变量的因子
            invole_v = []
            for idx, f in enumerate(F):
                if v in f.var_list:
                    invole_v.append(f)
                    del F[idx]
            # 将所有找到的因子相乘
            g = invole_v[0]
            for f in invole_v[1:]:
                g = g.multiply(f)
            # 相乘得到的新的因子加入F
            F.append(g)
        print("RESULT:")
        # 将所有剩余的因子相乘（这些因子只涉及查询变量）
        res = F[0]
        for f in F[1:]:
            res = res.multiply(f)
        total = sum(res.cpt.values())
        res.cpt={k:v/total for k, v in res.cpt.items()}
        res.printInf()
    @staticmethod
    def printFactors(F):
        for f in F:
            f.printInf()

B = Node('B', ['B'])
E = Node('E', ['E'])
A = Node('A', ['A', 'B', 'E'])
J = Node('J', ['J', 'A'])
M = Node('M', ['M', 'A'])

B.setCpt({'0': 0.999, '1': 0.001})
E.setCpt({'0': 0.998, '1': 0.002})
A.setCpt({'111': 0.95, '011': 0.05, '110':0.94, '010':0.06,
          '101': 0.29, '001': 0.71, '100':0.001,'000':0.999})
J.setCpt({'11':0.90, '01': 0.10, '10': 0.05, '00': 0.95})
M.setCpt({'11':0.70, '01': 0.30, '10': 0.01, '00': 0.99})



In [49]:
f = Node('A', ['A', 'B'])

In [50]:
g = Node('C', ['C', 'B'])

In [51]:
f.setCpt({'11': 0.9, '10':0.1, '01':0.4, '00': 0.6})

In [52]:
g.setCpt({'11':0.7, '01':0.3, '10':0.8, '00':0.2})

In [53]:
f.multiply(g).cpt

{'111': 0.63,
 '110': 0.27,
 '101': 0.08000000000000002,
 '100': 0.020000000000000004,
 '011': 0.27999999999999997,
 '010': 0.12,
 '001': 0.48,
 '000': 0.12}

In [56]:
f.sumout('A').cpt

{'1': 1.3, '0': 0.7}

In [54]:
f.restrict('A', '1').cpt

{'1': 0.9, '0': 0.1}